In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
driver.get('http://www.google.com')
driver.quit()

### request 테스트(한 페이지만 로드해서 정보 가져오기에 좋음_selenium보다 빠름)
- 페이지 내에서 날짜, 좌석등급 정보를 바꿔줘야 하기 때문에 이번 상황에는 부적절

In [ ]:
import requests

url = 'https://www.google.com/travel/flights/search?tfs=CBwQAhoqEgoyMDIzLTA5LTAxag4IAxIKL20vMDMwcWIzdHIMCAMSCC9tLzBoc3FmQAFIAXABggELCP___________wGYAQI&tfu=EgYIARABGAA'
response = requests.get(url)
response.status_code

200

### selenium(페이지 내에서 선택, 클릭 등이 필요할 경우 사용하기 좋음)

In [406]:
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By

options = Options()
# options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
driver.get('https://www.google.com/travel/flights/search?tfs=CBwQAhoqEgoyMDIzLTA5LTAxag4IAxIKL20vMDMwcWIzdHIMCAMSCC9tLzBoc3FmQAFIAXABggELCP___________wGYAQI&tfu=EgYIARABGAA')

time.sleep(1)

tckt = []
data = []
week = ['월', '화', '수', '목', '금', '토', '일']
d, j = 1, 4

for _ in range(30):
    for i in range(1,5):

        # 좌석등급 바꾸기
        gbox_path = '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[1]/div[3]/div/div/div'
        gbox = driver.find_element(By.XPATH, gbox_path)
        gbox.click()
        g_path = f'//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[1]/div[3]/div/div/div/div[2]/ul/li[{i}]'
        g = driver.find_element(By.XPATH, g_path)
        g.click()

        time.sleep(1)

        # 현재 페이지 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 항공권 정보 긁어오기
        ticket_box = soup.select('ul.Rk10dc')
        tickets = ticket_box[0].select('li>div>div.yR1fYc>div>div:nth-of-type(2)')
        tickets.extend(ticket_box[1].select('li>div>div.yR1fYc>div>div:nth-of-type(2)'))

        # 좌석 등급
        grade = soup.select_one('span#i19').text
        # 날짜
        day = f'9월 {d}일 ({week[j]})'

        tckt = [day, grade]

        for one in tickets:
            for info in one.select('span, div'):
                if info.text == '':
                    pass
                elif info.text in tckt:
                    pass
                else:
                    tckt.append(info.text)

                if info.get('aria-label') is not None and info.get('aria-label').startswith('기착'):
                    tckt.append(info.get('aria-label'))
            data.append(tckt)
            tckt = [day, grade]

        time.sleep(1)


    # 날짜 바꾸기
    nd_path = '//*[@id="yDmH0d"]/c-wiz[2]/div/div[2]/c-wiz/div[1]/c-wiz/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div[1]/div/div[1]/div/div[3]'
    next_day = driver.find_element(By.XPATH, nd_path)
    next_day.click()
    
    d += 1
    j += 1

    if j > 6: j = 0

    time.sleep(1)

driver.quit()

### 좌석 등급 정보 가져오기, 날짜 정보 가져오기
- 날짜는 html 내에서 업데이트가 되지 않고 첫 페이지의 날짜로만 표시됨

In [279]:
grade = soup.select_one('span#i19').text
grade
# 날짜가 html상에서 업데이트 되지 않아 수동으로 업데이트
day = soup.select_one('input.TP4Lpb.eoY5cb.j0Ppje').get('value')
day

'9월 1일 (금)'

- 수동으로 페이지가 바뀔 때마다 날짜가 입력되도록 함

In [ ]:
week = ['월', '화', '수', '목', '금', '토', '일']
i, j = 1, 4
for _ in range(10):
    if j > 6: j = 0

    day = f'9월 {i}일 ({week[j]})'
    i += 1
    j += 1

    print(day)

### ticket정보 list 저장
- ticket정보를 리스트로 저장하는 코드를 테스트 해보는 코드

In [573]:
# ticket_box = soup.select('ul.Rk10dc')
# tickets = ticket_box[0].select('li>div>div.yR1fYc>div>div:nth-of-type(2)')
# tickets.extend(ticket_box[1].select('li>div>div.yR1fYc>div>div:nth-of-type(2)'))

# 티켓별 정보를 tckt 리스트로 불러와 data 리스트 안에 넣어줌
# data = []
# tckt = [day, grade]

# for one in tickets:
#     for info in one.select('span, div'):
#         if info.text not in tckt:
#             tckt.append(info.text)
#         elif info.text == '':
#             pass
#         else:
#             pass
#     data.append(tckt)
#     tckt = [day, grade]

- 기착시간 가져오기

In [333]:
tickets[9].select('div')
for i in tickets[9].select('div'):
    if i.get('aria-label') is not None and i.get('aria-label').startswith('기착'):
        print(i.get('aria-label'))

기착 시간(1/2)은 샌프란시스코의 샌프란시스코 국제공항에서 5시간입니다. 기착 시간(2/2)은 도쿄도의 도쿄 국제공항에서 4시간 30분입니다.


### 데이터프레임으로 만들고 정제

- 여러 가지 시도를 위해 깊은 복사를 해준다.

In [428]:
import copy
data1 = copy.deepcopy(data)

- 정보를 추출하는 과정에서 제대로 추출되지 않는 경우를 찾아 해당 인덱스의 항목 내용을 확인해 봄.

In [352]:
data1[0]
data1[4][30:]
data1[9][30:]

['야간 기착',
 'OAK에서 SJC(으)로 환승',
 '공항 변경',
 '기착 시간(1/2)은 산호세에서 야간에 9시간 55분입니다. 오클랜드 국제공항발 노르만 Y. 미네타 산호세 국제공항행은 여기에서 환승합니다. 기착 시간(2/2)은 호놀룰루의 호놀룰루 국제공항에서 3시간 55분입니다.',
 'CO2 934kg+23% 배출',
 'CO2 934kg',
 '+23% 배출',
 '이 항공편 가격에는 기내 선반 이용 요금이 포함되지 않습니다. 기내 수하물이 필요한 경우 수하물 필터를 사용하여 가격을 업데이트하세요.₩940,554',
 '이 항공편 가격에는 기내 선반 이용 요금이 포함되지 않습니다. 기내 수하물이 필요한 경우 수하물 필터를 사용하여 가격을 업데이트하세요.',
 '수하물 필터',
 '₩940,554']

- 각 정보 추출하는 코드를 테스트

In [ ]:
# # 경유 횟수 추출('₩'로 바꿔주어 가격 추출 가능)
# for i in data1:
#     for j in i:
#         if j.startswith('경유') and len(j)==5:
#             print(j[-2])
#         elif j.startswith('직항'):
#             print(0)

# # 소요 시간 추출
# for i in data1:
#     for j in i:
#         if j[:2].isdigit() and len(j) < 9:
#             print(j)

# # 출발.도착.환승 공항 추출
# import re
# for i in data1:
#     for j in i:
#         if (re.match("^[a-zA-Z]+.*", j) and len(j) == 3):
#             print(j)
#         if (re.match("^[a-zA-Z]+.*", j) and j.endswith('환승')):
#             print(j[:3],j[6:9])

# # 항공사 추출(11번째 항목에 따라 항공사 정보를 담은 인덱스가 달라짐)
# for i in data1:
#     if i[11].startswith('함께'):
#         print(i[15])
#     else:
#         print(i[11])

# # 날짜와 요일 정보를 따로 불러오기
# for i in data1:
#     print(i[0][:-4])  # 날짜
#     print(i[0][-2:-1])  # 요일

# # 기착 시간 불러오기
# import re
# for i in data1:
#     for j in i:
#         if j.startswith('기착'):
#             j = j.split('. ')
#             for k in j:
#                 if k.startswith('기착'):
#                     numbers = re.findall(r'[0-9]{1,2}',k[12:])  # {1,2} -> 자릿수
#                     if len(numbers) == 1:
#                         print(f"{numbers[0]}:00")  # 기착 시간(시간)
#                     elif len(numbers) == 2:
#                         print(f"{numbers[0]}:{numbers[1]}")  # 기착 시간(시간+분)
#             print('\n')

# # 도착 날짜,요일,시간
# for i in data1:
#     j = i[10].split(' ')
#     print(j[0],j[1])
#     print(j[2][1:-2])
#     print(j[3],j[4])

- 최종 실행 코드
    - 아래 코드를 실행하여 새로운 리스트에 다시 정리해준다.

In [ ]:
import re

one_t = []
data_li = []

for i in data1:
    one_t.append(i[0][:-4])  # 출발 날짜
    one_t.append(i[0][-2:-1])  # 출발 요일
    one_t.append(i[5])  # 출발 시간
    one_t.append(i[1])  # 좌석 등급

    arrival = i[10].split(' ')  # 도착 정보
    one_t.append(arrival[0] + ' ' + arrival[1])  # 도착 날짜
    one_t.append(arrival[2][1:-2])  # 도착 요일
    one_t.append(arrival[3] + ' ' + arrival[4])  # 도착 시간

    for j in i:
        if j.startswith('경유') and len(j)==5:
            one_t.append(j[-2])  # 경유 횟수
        elif j.startswith('직항'):
            one_t.append(0)  # 직항
        elif j.startswith('₩'):
            one_t.append(j[1:])  # 가격
        elif j[:2].isdigit() and (j.endswith('분') or j.endswith('시간')):
            one_t.append(j)  # 소요 시간

    # 항공사
    if i[11].startswith('함께'):
        one_t.append(i[15])
    else:
        one_t.append(i[11])

    for j in i:
        if (re.match("^[a-zA-Z]+.*", j) and len(j) == 3):
            if j in ['GMP','ICN','LAX','HUR']:
                one_t.append(j)# 출발.도착 공항
            else:
                pass
        # elif (re.match("^[a-zA-Z]+.*", j) and j.endswith('환승')):
        #     one_t.append(f'환승 도착 {j[:3]}')# 환승 도착 공항
        #     one_t.append(f'환승 출발 {j[6:9]}')# 환승 출발 공항

    # for j in i:
    #     if j.startswith('기착'):
    #         j = j.split('. ')
    #         for k in j:
    #             if k.startswith('기착'):
    #                 numbers = re.findall(r'[0-9]{1,2}',k[12:])
    #                 if len(numbers) == 1:
    #                     one_t.append(f"{numbers[0]}:00")  # 기착 시간(시간)
    #                 elif len(numbers) == 2:
    #                     one_t.append(f"{numbers[0]}:{numbers[1]}")  # 기착 시간(시간+분)

    data_li.append(one_t)
    one_t = []

data_li

- 정리한 리스트를 데이터 프레임 형태로 저장

In [515]:
import pandas as pd
columns = ['departure_date','departure_day','departure_time','class','arrival_date','arrival_day','arrival_time','flight_time','flight_type','price','airlines','port_d','port_a']
df = pd.DataFrame(data_li, columns=columns)

- df 파일 복사

In [527]:
df1 = df.copy()

- 원하는 형식에 맞게 바꾸고 csv 파일로 내보냄

In [ ]:
# # 출발 날짜 형식 09-01 형태로 바꿔주기
# for i in range(len(df1['departure_date'])):
#     if len(df1['departure_date'][i]) == 5:
#         month = int(df1['departure_date'][i][0])
#         day = int(df1['departure_date'][i][3])
#     else:
#         month = int(df1['departure_date'][i][0])
#         day = int(df1['departure_date'][i][3:5])

#     df1['departure_date'][i] = f"{month:02}-{day:02}"



# # 도착 날짜 형식 09-01 형태로 바꿔주기
# for i in range(len(df1['arrival_date'])):
#     if len(df1['arrival_date'][i]) == 5:   # 9월 3일
#         month = int(df1['arrival_date'][i][0])
#         day = int(df1['arrival_date'][i][3])
#     elif len(df1['arrival_date'][i]) == 6 and df1['arrival_date'][i][1].isdigit():  # 10월 1일
#         month = int(df1['arrival_date'][i][:2])
#         day = int(df1['arrival_date'][i][4])
#     else:  # 9월 30일
#         month = int(df1['arrival_date'][i][0])
#         day = int(df1['arrival_date'][i][3:5])

#     df1['arrival_date'][i] = f"{month:02}-{day:02}"




# # 소요 시간 형식 00:00 형태로 바꿔주기
# for i in range(len(df1['flight_time'])):
#     if len(df1['flight_time'][i]) == 4:
#         hour = int(df1['flight_time'][i][0:2])
#     elif len(df1['flight_time'][i]) == 7:
#         hour = int(df1['flight_time'][i][0:2])
#         minutes = int(df1['flight_time'][i][5])
#     else:
#         hour = int(df1['flight_time'][i][0:2])
#         minutes = int(df1['flight_time'][i][5:7])

#     df1['flight_time'][i] = f"{hour:02}:{minutes:02}"

In [532]:
df1
# df1.to_csv("LAtoKR.csv")

- 저장된 csv파일 확인

In [533]:
pd.read_csv('LAtoKR.csv', index_col=0)

,departure_date,departure_day,departure_time,class,arrival_date,arrival_day,arrival_time,flight_time,flight_type,price,airlines,port_d,port_a
0,09-01,금,오후 5:00,이코노미,09-03,일,오후 12:45,27:45,1,"868,975",중화항공,LAX,GMP
1,09-01,금,오후 12:50,이코노미,09-03,일,오전 5:50,25:45,1,"920,168",필리핀항공,LAX,ICN
2,09-01,금,오후 12:15,이코노미,09-03,일,오전 11:00,30:45,1,"981,879",에바항공아시아나,LAX,ICN
3,09-01,금,오전 10:50,이코노미,09-02,토,오후 3:45,12:55,0,"1,136,994",Air Premia Inc,LAX,ICN
4,09-01,금,오후 12:40,이코노미,09-02,토,오후 5:35,12:55,0,"1,202,299",아시아나,LAX,ICN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30988,09-30,토,오후 7:22,퍼스트,10-02,월,오후 10:25,35:03,2,"23,621,801","유나이티드항공, 일본항공, 아시아나아메리칸항공, 전일본공수운항사: Skywest D...",GMP,NaN
30989,09-30,토,오전 12:30,퍼스트,10-01,일,오후 5:40,25:10,1,"26,333,945","캐세이패시픽항공, 아시아나",LAX,ICN
30990,09-30,토,오후 11:30,퍼스트,10-02,월,오후 5:40,26:10,1,"26,633,526","캐세이패시픽항공, 아시아나",LAX,ICN
30991,09-30,토,오전 12:30,퍼스트,10-02,월,오전 7:10,38:40,2,"28,177,872","캐세이패시픽항공, 말레이항공",LAX,ICN
